# <center>Analisando dados BIDMC PPG - Respiração</center>

Os dados originais (https://physionet.org/content/bidmc/1.0.0/) foram adquiridos de pacientes criticamente enfermos durante atendimento hospitalar no Beth Israel Deaconess Medical Center (Boston, MA, EUA). Dois anotadores anotaram manualmente respirações individuais em cada registro usando o sinal respiratório de impedância. As 53 gravações dentro do conjunto de dados, cada uma com 8 minutos de duração, cada uma contém:
<ul>
<li> Sinais fisiológicos, como PPG, sinal respiratório de impedância e eletrocardiograma (ECG). Eles são amostrados em 125 Hz; </li>
<li> Parâmetros fisiológicos, como frequência cardíaca (FC), frequência respiratória (FR) e nível de saturação de oxigênio no sangue (SpO2). Eles são amostrados em 1 Hz; </li>
<li> Parâmetros fixos, como idade e sexo; </li>
<li> Anotações manuais de respirações.</li>
</ul>

## Dados Formato CSV

Arquivos CSV separados são fornecidos para cada gravação (onde ## é o número do paciente), contendo:
<ul>
<li> bidmc _ ## _ Breaths.csv: anotações de respiração manual; </li>
<li> bidmc _ ## _ Signals.csv: sinais fisiológicos; </li>
<li> bidmc _ ## _ Numerics.csv: Parâmetros fisiológicos; </li>
<li> bidmc _ ## _ Fix.txt: variáveis fixas.</li>
</ul>

In [ ]:
%%time
import pandas as pd
import os
import plotly
import plotly.offline as py
import plotly.graph_objs as go
import numpy  as np
import h5py

from scipy.fft import fft, fftfreq, ifft
from datetime import datetime
from plotly.subplots import make_subplots
from scipy.signal import find_peaks
from scipy.interpolate import interp1d
from openpyxl import load_workbook
from PyEMD import EMD
from scipy.signal import butter, lfilter, freqz

#pasta para salvar os arquivos
os.chdir("C:\\Users\\ayalo\\PPG\\RESULTADOS\\BIDMC" )

# Inicializa o modo Plotly Notebook 
py.init_notebook_mode(connected=False)

In [ ]:
#Data BIDMC
#60000 amostras
num_arquivos = 53
fre_sinal = 125

#dados manipulaveis
frequencia = 125
inicio = 60
fim = 100
janela = 20

amostras1 = inicio * fre_sinal
amostras = fim * fre_sinal
N = amostras - amostras1

N_PPG = (480 * frequencia)
N_JANELA = janela * frequencia
N_JANELA_NORMAL = janela * fre_sinal
janela_tempo = fim - (inicio + janela) #quantidade de repetiçoes

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

def manipulation_frequency(array):
    array = np.asarray(array)
    vetor = np.zeros(N_PPG)
    for i in range(N_PPG):
        vetor[i] = array[i*int(fre_sinal/frequencia)]
    return vetor 

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a


In [ ]:
%%time

#Cria Matriz Vazia
PPG_bid = np.empty((num_arquivos, N_PPG))
IMP_bid = np.empty((num_arquivos, 60000))
rpm_bid = np.empty((num_arquivos, 480))


for i in range(num_arquivos):
    caminhoBIDMC  = "C:\\Users\\ayalo\\PPG\\DADOS\\bidmc_csv\\bidmc_"+str(i+1)+"_Signals.csv" 
    data = pd.read_csv(caminhoBIDMC)
    data = data[[' PLETH']]
    data = np.array(data)
    data = data.transpose()
    data = data[0][0:60000]
    PPG_bid[i] = manipulation_frequency(data)
    
    data = pd.read_csv(caminhoBIDMC)
    data = data[[' RESP']]
    data = np.array(data)
    data = data.transpose()
    IMP_bid[i] = data[0][0:60000]
    
    caminhoBIDMC  = "C:\\Users\\ayalo\\PPG\\DADOS\\bidmc_csv\\bidmc_"+str(i+1)+"_Numerics.csv" 
    data = pd.read_csv(caminhoBIDMC)
    data = data[[' RESP']]
    data = np.array(data)
    data[np.isnan(data)] = 0
    data = data.transpose()
    rpm_bid[i] = data[0][0:480] 

In [ ]:
%%time
resultado33 = np.empty((num_arquivos,5*janela_tempo))
www = 0

for kk in range(janela_tempo):
    ######################################################################### FFT ##########################################################################################################################
    # sample spacing
    T = 1 / fre_sinal
    T_PPG = 1 / frequencia

    Xfft_PPG = np.empty((num_arquivos, int(N_JANELA/2)))
    Yfft_PPG = np.empty((num_arquivos, int(N_JANELA/2)))
    Xfft_IMP = np.empty((num_arquivos, int(N_JANELA_NORMAL/2)))
    Yfft_IMP = np.empty((num_arquivos, int(N_JANELA_NORMAL/2)))
    resultado = np.empty((num_arquivos,6))
    sinal_ppg = np.empty((num_arquivos,N_JANELA))

    FrPPG = np.array(range(num_arquivos))
    FrIMP = np.array(range(num_arquivos))
    FrREF = np.array(range(num_arquivos))
    Erro_fft = np.array(range(num_arquivos))
    Erro_medido = np.array(range(num_arquivos))

    Y = np.linspace(1, 42, num_arquivos)
    for i in range(num_arquivos):
        
        cp1 = (kk * frequencia) + (frequencia * inicio) 
        cp2 = N_JANELA + cp1
        
        #################################################################################fft
        y0 = PPG_bid[i][cp1:cp2]
        #################################################################################fft
        
        
        
        
#         #################################################################################linha de base/amplitude/frequencia      
#         #sinal original com aumento das extremidades e remoção do sinal DC por media
#         sinal_original = PPG_bid[i][cp1-2000:cp2+2000]
#         sinal_ac = sinal_original - np.mean(sinal_original)
#         #pegando os picos e vales
#         val_x_peaks_ac, _ = find_peaks(sinal_ac, prominence=0.1)
#         vales_x, _ = find_peaks((sinal_ac*(-1)), prominence=0.1)
#         #Diferença no numero de vales e picos é menor q 2
#         if( abs(len(val_x_peaks_ac) - len(vales_x)) < 3):
            
# #             ###############################AMPLITUDE    
# #             #quantidade de picos menores/igual e primeiro pico começa antes do vale
# #             if( (len(val_x_peaks_ac) <= len(vales_x)) and ( val_x_peaks_ac[0] <  vales_x[0]) ):
# #                 pedro = np.zeros(len(val_x_peaks_ac)-1)
# #                 for z in range(len(pedro)-1):
# #                     pedro[z] = sinal_ac[val_x_peaks_ac[z+1]] - sinal_ac[vales_x[z]]
                
# #             #quantidade de picos menores/igual e primeiro pico começa depois do vale
# #             elif( (len(val_x_peaks_ac) <= len(vales_x)) and ( val_x_peaks_ac[0] >  vales_x[0]) ):
# #                 pedro = np.zeros(len(val_x_peaks_ac))
# #                 for z in range(len(pedro)):
# #                     pedro[z] = sinal_ac[val_x_peaks_ac[z]] - sinal_ac[vales_x[z]]

# #             #quantidade de vales menores/igual e primeiro pico começa antes do vale
# #             elif( (len(vales_x) <= len(val_x_peaks_ac)) and ( val_x_peaks_ac[0] <  vales_x[0]) ):
# #                 pedro = np.zeros(len(vales_x)-1)
# #                 for z in range(len(pedro)-1):
# #                     pedro[z] = sinal_ac[val_x_peaks_ac[z+1]] - sinal_ac[vales_x[z]]

# #             #quantidade de vales menores/igual e primeiro pico começa depois do vale
# #             elif( (len(vales_x) <= len(val_x_peaks_ac)) and ( val_x_peaks_ac[0] >  vales_x[0]) ):
# #                 pedro = np.zeros(len(vales_x))
# #                 for z in range(len(pedro)):
# #                     pedro[z] = sinal_ac[val_x_peaks_ac[z]] - sinal_ac[vales_x[z]]

# #             else:
# #                 www = www + 1
# #                 print("condiçao maluca ver", inicio, fim, www)


# #             s_a = interp1d(vales_x[0:len(pedro)], pedro, kind = 'linear',bounds_error = False, fill_value=0.0)
# #             maria = np.zeros(len(sinal_original))
# #             for aux3 in range(len(sinal_original)):
# #                 maria[aux3] = s_a(aux3)    
# #             y0 = maria[2000:(len(sinal_original)-2000)]
# #             ###############################AMPLITUDE
            
#             # ###############################LINHA DE BASE            
#             # l_b = interp1d(val_x_peaks_ac,sinal_original[val_x_peaks_ac], kind = 'linear',bounds_error = False, fill_value=0.0)
#             # pedro = np.zeros(len(sinal_original))
#             # for k in range(len(sinal_original)):
#             #     pedro[k] = l_b(k)
#             # y0 = pedro[2000:(len(sinal_original)-2000)]
#             # ###############################LINHA DE BASE
            
#             ###############################FREQUENCIA
#             fm_y = np.zeros(len(val_x_peaks_ac)-1)
#             for z in range(len(val_x_peaks_ac)-1):
#                 fm_y[z] = val_x_peaks_ac[z+1]-val_x_peaks_ac[z]
#             fm_x = val_x_peaks_ac[0:(len(val_x_peaks_ac)-1)]
#             s_f = interp1d(fm_x,fm_y, kind = 'linear',bounds_error = False, fill_value=0.0)
#             joao = np.zeros(len(sinal_original))
#             for k in range(len(sinal_original)):
#                 joao[k] = s_f(k)
#             y0 = joao[2000:(len(sinal_original)-2000)]
#             ###############################FREQUENCIA            
            
#         else:
#             www = www + 1
#             print("picos e vales diferentes de 1!", cp1, cp2, www, i)
#             tt = np.linspace(0, janela, N_JANELA, endpoint=False)
#             y0 = np.sin(0.83*2*np.pi*tt)
        
#         #############EMD
#         order = 5       
#         cutoff = 2
#         data = PPG_bid[i][cp1:cp2]
#         #ROGERIN = butter_lowpass_filter(data, cutoff, frequencia, order)
#         time = np.linspace(0, 1, len(data))
#         emd1 = EMD()
#         IMF = emd1.emd(data, time)

#         #y0 = IMF[0] #IMF1
#         #y0 = IMF[1] #IMF2
#         #y0 = IMF[2] #IMF3
#         #y0 = IMF[3] #IMF4
#         #############EMD
        
        sinal_ppg[i] = y0
        yf0 = fft(y0)
        xf0 = fftfreq(N_JANELA, T_PPG)[:N_JANELA//2]
        yf0 = np.abs(yf0[0:N_JANELA//2])
        Xfft_PPG[i] = xf0
        Yfft_PPG[i] = yf0

        cp3 = (kk * fre_sinal) + (fre_sinal * inicio)
        cp4 = N_JANELA_NORMAL + cp3
        y1 = IMP_bid[i][cp3:cp4]
        yf1 = fft(y1)
        x1 = fftfreq(N_JANELA_NORMAL, T)[:N_JANELA_NORMAL//2]
        y1 = np.abs(yf1[0:N_JANELA_NORMAL//2])
        Xfft_IMP[i] = x1
        Yfft_IMP[i] = y1

        fyy = Xfft_PPG[i][5] - Xfft_PPG[i][4]
        p1 = int(0.15/fyy)
        p2 = int(0.83/fyy)
        
        fy = Xfft_IMP[i][5] - Xfft_IMP[i][4]
        p3 = int(0.15/fy)
        p4 = int(0.83/fy)     

        pos_y = Yfft_PPG[i][p1:p2].argmax()
        val_x = Xfft_PPG[i][pos_y+p1]
        FrPPG[i] = val_x*60

        pos_y = Yfft_IMP[i][p3:p4].argmax()
        val_x = Xfft_IMP[i][pos_y+p3]
        FrIMP[i] = val_x*60

        FrREF[i] = np.mean(rpm_bid[i][(inicio+kk):(fim+kk)])

        Erro_fft[i] = FrIMP[i] - FrPPG[i]

        Erro_medido[i] = FrREF[i] - FrPPG[i]
        Y[i] = i + 1
        resultado[i] = [ Y[i], FrPPG[i], FrIMP[i], FrREF[i], Erro_fft[i], Erro_medido[i] ] 
    print(cp1,cp2)
    print(cp3,cp4)
    ########################################################################## Calcula raiz do erro medio quadradico ###########################################################################################

    resultado1 = np.empty((5,2))

    RMSE_fftIMP = rmse(FrPPG, FrIMP)
    RMSE_FrREF = rmse(FrPPG, FrREF)

    percentual0_fftIMP = ((np.count_nonzero((Erro_fft==0)))*100)/Erro_fft.size
    percentual1_fftIMP = ((np.count_nonzero((Erro_fft<1)&(Erro_fft>-1)))*100)/Erro_fft.size
    percentual2_fftIMP = ((np.count_nonzero((Erro_fft<2)&(Erro_fft>-2)))*100)/Erro_fft.size
    percentual3_fftIMP = ((np.count_nonzero((Erro_fft<3)&(Erro_fft>-3)))*100)/Erro_fft.size

    percentual0_FrREF = (np.count_nonzero((Erro_medido==0))*100)/Erro_medido.size
    percentual1_FrREF = ((np.count_nonzero((Erro_medido<1)&(Erro_medido>-1)))*100)/Erro_medido.size
    percentual2_FrREF = ((np.count_nonzero((Erro_medido<2)&(Erro_medido>-2)))*100)/Erro_medido.size
    percentual3_FrREF = ((np.count_nonzero((Erro_medido<3)&(Erro_medido>-3)))*100)/Erro_medido.size

    resultado1[0] = [round(RMSE_fftIMP,1),round(RMSE_FrREF,1)]
    resultado1[1] = [round(percentual0_fftIMP,1),round(percentual0_FrREF,1)]
    resultado1[2] = [round(percentual1_fftIMP,1),round(percentual1_FrREF,1)]
    resultado1[3] = [round(percentual2_fftIMP,1),round(percentual2_FrREF,1)]
    resultado1[4] = [round(percentual3_fftIMP,1),round(percentual3_FrREF,1)]

    ###################################################################################### plot ################################################################################################################

    linhas = 5

    # Create figure
    fig = make_subplots(rows=linhas, cols=1)
    # Add all traces of pacient
    for i in range(num_arquivos):
        fig.add_scatter(y=sinal_ppg[i],
                        row=1, col=1, visible=False,
                        showlegend = False,
                        name = "(x,y)",
                        marker_color="red")
    for i in range(num_arquivos):
        fig.add_scatter(y=IMP_bid[i][cp3:cp4],
                        row=2, col=1, visible=False,
                        showlegend = False,
                        name = "(x,y)",
                        marker_color="black")
    for i in range(num_arquivos):
        fig.add_scatter(y=rpm_bid[i],
                        row=3, col=1, visible=False,
                        showlegend = False,
                         mode="markers",
                        name = "(x,y)",
                        marker_color="green")
    for i in range(num_arquivos):
        Y=Yfft_PPG[i] * 2/N_JANELA
        fig.add_scatter(x=Xfft_PPG[i][p1:p2], y=Y[p1:p2],
                        row=4, col=1, visible=False,
                        showlegend = False,
                        name = "(x,y)",
                        marker_color="red")
    for i in range(num_arquivos):
        Y=Yfft_IMP[i] * 2/N_JANELA_NORMAL
        fig.add_scatter(x=Xfft_IMP[i][p3:p4] , y=Y[p3:p4],
                        row=5, col=1, visible=False,
                        showlegend = False,
                        name = "(x,y)",
                        marker_color="black")

    # Make 10th trace visible
    for i in range(linhas):
        fig.data[i*num_arquivos].visible = True


    # Create and add slider
    steps = []
    for i in range(num_arquivos):
        step = dict(
            method="update",
            args=[{"visible": [False] * len(fig.data)}],  # layout attribute
        )
        for k in range(linhas):
            step["args"][0]["visible"][i+(k*num_arquivos)] = True  # Toggle i'th trace to "visible"
        steps.append(step)

    sliders = [dict(
        steps=steps
    )]

    # Update xaxis properties
    fig.update_xaxes(title_text="Amostras (u)", row=2, col=1)
    fig.update_xaxes(title_text="Tempo (s)", row=3, col=1)
    fig.update_xaxes(title_text="frequencia (Hz)", row=5, col=1)
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)

    # Update yaxis properties
    fig.update_yaxes(title_text="PPG", row=1, col=1)
    fig.update_yaxes(title_text="IMP", row=2, col=1)
    fig.update_yaxes(range = [1,50],title_text="rpm", row=3, col=1)
    fig.update_yaxes(title_text="FFT-PPG", row=4, col=1)
    fig.update_yaxes(title_text="FFT-IMP", row=5, col=1)
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)

    fig.update_layout(
        title="Bidmc Sinal:",
        sliders=sliders,
        plot_bgcolor='rgba(0,0,0,0)',
        height=1000
    )

    # Edit slider labels
    fig['layout']['sliders'][0]['currentvalue']['prefix']='Paciente: '
    for i in range(num_arquivos):
        fig['layout']['sliders'][0]['steps'][i]['label']=i+1


    #fig.show()
    plotly.offline.plot(fig, filename="De"+str(inicio+kk)+"ate"+str(inicio+janela+kk)+'.html',auto_open=False)


    ######################################################################################## salva excel #######################################################################################################

#     data_array = resultado[:,1:]
#     row_indices = resultado[:,0]
#     column_names = ['PPG','IMP', 'RR', 'IMP-PPG', 'RR-PPG']


#     data_df = pd.DataFrame(data_array, columns=column_names, index=row_indices)

#     book = load_workbook('erro.xlsx')
#     writer = pd.ExcelWriter('erro.xlsx', engine = 'openpyxl', mode='a')
#     writer.book = book
#     data_df.to_excel(writer, sheet_name="De"+str(inicio+kk)+"ate"+str(inicio+janela+kk))
#     writer.close()

    data_array = resultado1
    row_indices = ['RMSE','PERCENTUAL-0rpm','PERCENTUAL-1rpm','PERCENTUAL-2rpm','PERCENTUAL-3rpm']
    column_names = ['IMP-PPG', 'RR-PPG']


    data_df = pd.DataFrame(data_array, columns=column_names, index=row_indices)

    book = load_workbook('precisao.xlsx')
    writer = pd.ExcelWriter('precisao.xlsx', engine = 'openpyxl', mode='a')
    writer.book = book
    data_df.to_excel(writer, sheet_name="De"+str(inicio+kk)+"ate"+str(inicio+janela+kk))
    writer.close()
    
    resultado33[0:,(kk*5):(5+(kk*5))] = resultado[:,1:]
    
    print(kk)
    
data_array = resultado33
row_indices = resultado[:,0]
data_df = pd.DataFrame(data_array, index=row_indices)

book = load_workbook('erro.xlsx')
writer = pd.ExcelWriter('erro.xlsx', engine = 'openpyxl', mode='a')
writer.book = book
data_df.to_excel(writer, sheet_name="TUDO")
writer.close()

In [ ]:
sinal_original = PPG_bid[1][cp1-375:cp2+375]
val_x_peaks, _ = find_peaks(sinal_original, prominence=1)
vales_x, _ = find_peaks((sinal_original*(-1)), prominence=1)
#l_b = interp1d(val_x_peaks,sinal_original[val_x_peaks], kind = 'linear',bounds_error = False, fill_value=0.0)
#pedro = np.zeros(len(sinal_original))
#for k in range(len(sinal_original)):
#    pedro[k] = l_b(k)
#y0 = pedro[375:(len(sinal_original)-375)]
val_x_peaks

In [ ]:
order = 8       
cutoff = 5
data = PPG_bid[46][7500-375:10000+375]
ROGERIN = butter_lowpass_filter(data, cutoff, frequencia, order)

plt.figure(figsize=(15, 4))
plt.plot(ROGERIN)
plt.show()
len(ROGERIN)

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt 
from scipy.signal import find_peaks

sinal_original = ROGERIN#PPG_bid[46][7500-375:10000+375]
xx = sinal_original - np.mean(sinal_original)
peaks, _ = find_peaks(xx, distance=20)
peaks2, _ = find_peaks(xx, prominence=0.1)      # BEST!
peaks3, _ = find_peaks(xx, width=20)
peaks4, _ = find_peaks(xx, threshold=0.4)     # Required vertical distance to its direct neighbouring samples, pretty useless
#plt.subplot(2, 2, 1)
#plt.plot(peaks, xx[peaks], "xr"); plt.plot(xx); plt.legend(['distance'])
#plt.subplot(2, 2, 2)
plt.figure(figsize=(15, 4))
plt.plot(peaks2, xx[peaks2], "ob"); plt.plot(xx); plt.legend(['prominence'])


plt.show()

In [ ]:
val_x_peaks, _ = find_peaks(sinal_original, prominence=0.1)
l_b = interp1d(val_x_peaks,sinal_original[val_x_peaks], kind = 'linear',bounds_error = False, fill_value=0.0)
pedro = np.zeros(len(sinal_original))
print(len(sinal_original))
for k in range(len(sinal_original)):
    pedro[k] = l_b(k)
y0 = pedro[375:(len(sinal_original)-375)]

plt.figure(figsize=(15, 4))
plt.plot(val_x_peaks,sinal_original[val_x_peaks])


plt.show()

In [ ]:
T_PPG = 1/125
y0 = PPG_bid[52][7500-375:10000+375]
N_JANELA = len(y0)

yf0 = fft(y0)
xf0 = fftfreq(N_JANELA, T_PPG)[:N_JANELA//2]
yf0 = np.abs(yf0[0:N_JANELA//2])

plt.figure(figsize=(15, 4))
plt.plot(xf0,yf0)


plt.show()

In [ ]:
order = 8       
cutoff = 2
data = PPG_bid[52][0:12500]
ROGERIN = butter_lowpass_filter(data, cutoff, frequencia, order)


In [ ]:
# Create figure
fig = make_subplots(rows=1, cols=1)

    

fig.add_scatter(y=data, name = "(x,y)", marker_color="red",row=1, col=1, showlegend = False)
fig.add_scatter(y=ROGERIN, name = "(x,y)", marker_color="black",row=1, col=1, showlegend = False)


# Make 10th trace visible
fig.data[0].visible = True

# Update xaxis properties
fig.update_xaxes(title_text="Respirações (rpm)", row=3, col=1)


# Update yaxis properties
fig.update_yaxes(title_text="IMF5", row=1, col=1)
fig.update_yaxes(title_text="IMF6", row=2, col=1)
fig.update_yaxes(title_text="IMF7", row=3, col=1)


fig.update_layout(
    title="Valores da Frequencia respiratoria obtidos no sinal de PPG:",
    plot_bgcolor='rgba(0,0,0,0)',
    height=600
)

fig.show()

In [ ]:
%%time
#FFT
# Number of sample points
N = amostras
# sample spacing
T = 1 / 125

Xfft_PPG = np.empty((num_arquivos, int(amostras/2)))
Yfft_PPG = np.empty((num_arquivos, int(amostras/2)))
Xfft_IMP = np.empty((num_arquivos, int(amostras/2)))
Yfft_IMP = np.empty((num_arquivos, int(amostras/2)))

FrPPG = np.array(range(num_arquivos))
FrIMP = np.array(range(num_arquivos))
FrREF = np.array(range(num_arquivos))
Erro_fft = np.array(range(num_arquivos))
Erro_medido = np.array(range(num_arquivos))

for i in range(num_arquivos):
    y0 = PPG_bid[i]
    yf0 = fft(y0)
    xf0 = fftfreq(N, T)[:N//2]
    yf0 = np.abs(yf0[0:N//2])
    Xfft_PPG[i] = xf0
    Yfft_PPG[i] = yf0
        
    y1 = IMP_bid[i]
    yf1 = fft(y1)
    x1 = fftfreq(N, T)[:N//2]
    y1 = np.abs(yf1[0:N//2])
    Xfft_IMP[i] = x1
    Yfft_IMP[i] = y1
    
    fy = Xfft_IMP[i][5] - Xfft_IMP[i][4]
    p1 = int(0.15/fy)
    p2 = int(0.83/fy)
    
    pos_y = Yfft_PPG[i][p1:p2].argmax()
    val_x = Xfft_PPG[i][pos_y+p1]
    FrPPG[i] = val_x*60
    
    pos_y = Yfft_IMP[i][p1:p2].argmax()
    val_x = Xfft_IMP[i][pos_y+p1]
    FrIMP[i] = val_x*60
    
    FrREF[i] = np.mean(rpm_bid[i])
    
    Erro_fft[i] = FrIMP[i] - FrPPG[i]
    
    Erro_medido[i] = FrREF[i] - FrPPG[i]

In [ ]:
%%time

linhas = 5

# Create figure
fig = make_subplots(rows=linhas, cols=1)
# Add all traces of pacient
for i in range(num_arquivos):
    fig.add_scatter(y=PPG_bid[i][0:amostras],
                    row=1, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")
for i in range(num_arquivos):
    fig.add_scatter(y=IMP_bid[i][0:amostras],
                    row=2, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="black")
for i in range(num_arquivos):
    fig.add_scatter(y=rpm_bid[i],
                    row=3, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="green")
for i in range(num_arquivos):
    Y=Yfft_PPG[i] * 2/N
    fig.add_scatter(x=Xfft_PPG[i][p1:p2], y=Y[p1:p2],
                    row=4, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")
for i in range(num_arquivos):
    Y=Yfft_IMP[i] * 2/N
    fig.add_scatter(x=Xfft_IMP[i][p1:p2] , y=Y[p1:p2],
                    row=5, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="black")


# Make 10th trace visible
for i in range(linhas):
    fig.data[i*num_arquivos].visible = True


# Create and add slider
steps = []
for i in range(num_arquivos):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)}],  # layout attribute
    )
    for k in range(linhas):
        step["args"][0]["visible"][i+(k*num_arquivos)] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    steps=steps
)]

# Update xaxis properties
fig.update_xaxes(title_text="Amostras (u)", row=2, col=1)
fig.update_xaxes(title_text="frequencia (Hz)", row=4, col=1)
fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)

# Update yaxis properties
fig.update_yaxes(title_text="PPG", row=1, col=1)
fig.update_yaxes(title_text="IMP", row=2, col=1)
fig.update_yaxes(title_text="FFT-PPG", row=3, col=1)
fig.update_yaxes(title_text="FFT-IMP", row=4, col=1)
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)

fig.update_layout(
    title="Bidmc Sinal:",
    sliders=sliders,
    plot_bgcolor='rgba(0,0,0,0)',
    height=1000
)

# Edit slider labels
fig['layout']['sliders'][0]['currentvalue']['prefix']='Paciente: '
for i in range(num_arquivos):
    fig['layout']['sliders'][0]['steps'][i]['label']=i+1


fig.show()


####################### TABLE ##############################

Y = np.linspace(1, num_arquivos, num_arquivos)
fig = go.Figure(data=[go.Table(header=dict(values=['Paciente','PPG','IMP', 'RR', 'IMP-PPG', 'RR-PPG'],fill_color='LightGrey', align='center'),
                               cells=dict(values=[Y, FrPPG, FrIMP, FrREF, Erro_fft, Erro_medido],  fill_color = ['LightGrey','white', 'white']))
                                ])
    
fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    height=600
)
fig.show()
# Escreve o gráfico em um arquivo html
#plotly.offline.plot(fig, filename='Chart_1'+'.html',auto_open=False)

In [ ]:
percentual = ((np.count_nonzero((Erro_fft<4)&(Erro_fft>-4)))*100)/Erro_fft.size
percentual

In [ ]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

print(rmse(FrPPG, FrIMP))

## FIM

In [ ]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
# Load Data
df = px.data.tips()
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("JupyterDash Demo"),
    dcc.Graph(id='graph'),
    html.Label([
        "colorscale",
        dcc.Dropdown(
            id='colorscale-dropdown', clearable=False,
            value='plasma', options=[
                {'label': c, 'value': c}
                for c in px.colors.named_colorscales()
            ])
    ]),
])
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("colorscale-dropdown", "value")]
)
def update_figure(colorscale):
    return px.scatter(
        df, x="total_bill", y="tip", color="size",
        color_continuous_scale=colorscale,
        render_mode="webgl", title="Tips"
    )
# Run app and display result inline in the notebook
app.run_server(mode="jupyterlab")
1+1

In [ ]:
# Code source: https://dash-bootstrap-components.opensource.faculty.ai/examples/simple-sidebar/
import dash
import dash_bootstrap_components as dbc
import dash_html_components as html
import dash_core_components as dcc
import plotly.express as px
from dash.dependencies import Input, Output
import pandas as pd

# data source: https://www.kaggle.com/chubak/iranian-students-from-1968-to-2017
# data owner: Chubak Bidpaa
df = pd.read_csv('https://raw.githubusercontent.com/Coding-with-Adam/Dash-by-Plotly/master/Bootstrap/Side-Bar/iranian_students.csv')

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])


# styling the sidebar
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "padding": "2rem 1rem",
    "background-color": "#f8f9fa",
}

# padding for the page content
CONTENT_STYLE = {
    "margin-left": "18rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}

sidebar = html.Div(
    [
        html.H2("Sidebar", className="display-4"),
        html.Hr(),
        html.P(
            "Number of students per education level", className="lead"
        ),
        dbc.Nav(
            [
                dbc.NavLink("Home", href="/", active="exact"),
                dbc.NavLink("Page 1", href="/page-1", active="exact"),
                dbc.NavLink("Page 2", href="/page-2", active="exact"),
            ],
            vertical=True,
            pills=True,
        ),
    ],
    style=SIDEBAR_STYLE,
)

content = html.Div(id="page-content", children=[], style=CONTENT_STYLE)

app.layout = html.Div([
    dcc.Location(id="url"),
    sidebar,
    content
])


@app.callback(
    Output("page-content", "children"),
    [Input("url", "pathname")]
)
def render_page_content(pathname):
    if pathname == "/":
        return [
                html.H1('Kindergarten in Iran',
                        style={'textAlign':'center'}),
                dcc.Graph(id='bargraph',
                         figure=px.bar(df, barmode='group', x='Years',
                         y=['Girls Kindergarten', 'Boys Kindergarten']))
                ]
    elif pathname == "/page-1":
        return [
                html.H1('Grad School in Iran',
                        style={'textAlign':'center'}),
                dcc.Graph(id='bargraph',
                         figure=px.bar(df, barmode='group', x='Years',
                         y=['Girls Grade School', 'Boys Grade School']))
                ]
    elif pathname == "/page-2":
        return [
                html.H1('High School in Iran',
                        style={'textAlign':'center'}),
                dcc.Graph(id='bargraph',
                         figure=px.bar(df, barmode='group', x='Years',
                         y=['Girls High School', 'Boys High School']))
                ]
    # If the user tries to reach a different page, return a 404 message
    return dbc.Jumbotron(
        [
            html.H1("404: Not found", className="text-danger"),
            html.Hr(),
            html.P(f"The pathname {pathname} was not recognised..."),
        ]
    )


if __name__=='__main__':
    app.run_server()

    
# https://youtu.be/ln8dyS2y4Nc

In [ ]:
## Column

In [ ]:
caminhoBIDMC = "C:\\Users\\ayalo\\PPG\\DADOS\\bidmc_csv\\bidmc_01_Numerics.csv"
caminhoCAPNOBASE = "C:\\Users\\ayalo\\PPG\\DADOS\\capnobase_csv\\0009_8min_signal.csv"
caminhoRREST = "C:\\Users\\ayalo\\PPG\\DADOS\\rrest-syn_csv\\rrest-syn001_data.csv"
pd.set_option("display.max_rows", None)

In [ ]:
numerics = pd.read_csv(caminhoRREST)
numerics.columns = ["a", "b"]

In [ ]:
numerics.columns

In [ ]:
%%time

#from numpy import empty

Sinal_Original = np.empty((192, 100000))
Sinal_Original_IMP = np.empty((192, 100000))

for i in range(192):
    
    
    if i>0 and i < 10:
        caminho3 = "C:\\Users\\ayalo\\PPG\\DADOS\\rrest-syn_csv\\rrest-syn00"+str(i)+"_data.csv"
       

        
        #Dados sinal Impedancia
        signals = pd.read_csv(caminho3)
        signals.columns = ["a", "b"]
        s = signals['a'][0:100000]
        Signal_IMP = s.to_numpy()
        Sinal_Original_IMP[i] = Signal_IMP
        
        #Dados sinal PPG
        signals = pd.read_csv(caminho3)
        signals.columns = ["a", "b"]
        s = signals['b'][0:100000]
        Signal_PPG = s.to_numpy()
        Sinal_Original[i] = Signal_PPG
 
        
        
    if i >= 10 and i<100:
        caminho3 = "C:\\Users\\ayalo\\PPG\\DADOS\\rrest-syn_csv\\rrest-syn0"+str(i)+"_data.csv"
       
        
        #Dados sinal Impedancia
        signals = pd.read_csv(caminho3)
        signals.columns = ["a", "b"]
        s = signals['a'][0:100000]
        Signal_IMP = s.to_numpy()
        Sinal_Original_IMP[i] = Signal_IMP
        
        #Dados sinal PPG
        signals = pd.read_csv(caminho3)
        signals.columns = ["a", "b"]
        s = signals['b'][0:100000]
        Signal_PPG = s.to_numpy()
        Sinal_Original[i] = Signal_PPG
        print(i)
        

    if i >= 100 and i<200:
        caminho3 = "C:\\Users\\ayalo\\PPG\\DADOS\\rrest-syn_csv\\rrest-syn"+str(i)+"_data.csv"
       
        
        #Dados sinal Impedancia
        signals = pd.read_csv(caminho3)
        signals.columns = ["a", "b"]
        s = signals['a'][0:100000]
        Signal_IMP = s.to_numpy()
        Sinal_Original_IMP[i] = Signal_IMP
        
        #Dados sinal PPG
        signals = pd.read_csv(caminho3)
        signals.columns = ["a", "b"]
        s = signals['b'][0:100000]
        Signal_PPG = s.to_numpy()
        Sinal_Original[i] = Signal_PPG
        
        

In [ ]:
%%time
## Sinal Impedancia e PPG
# Código necessário para executar o Plotly em uma "code cell"
configure_plotly_browser_state()


# Create figure
fig = make_subplots(rows=2, cols=1)
# Add all traces of pacient
for step in range(192):
    Y = Sinal_Original_IMP[step]
    fig.add_scatter(y=Y[0:7500],
                    row=1, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="gold")
for step in range(192):
    Y = Sinal_Original[step]
    fig.add_scatter(y=Y[0:7500],
                    row=2, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="black")

# Make 10th trace visible
fig.data[0].visible = True
fig.data[192].visible = True

# Create and add slider
steps = []
for i in range(192):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},],  
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][i+192] = True
    steps.append(step)

sliders = [dict(
    steps=steps
)]

# Update xaxis properties
fig.update_xaxes(title_text="segundos (s)", row=2, col=1)

# Update yaxis properties
fig.update_yaxes(title_text="IMPEDANCIA", row=1, col=1)
fig.update_yaxes(title_text="PPG", row=2, col=1)

fig.update_layout(
    title="Sinais de Impedancia e PPG:",
    sliders=sliders,
    plot_bgcolor='rgba(0,0,0,0)',
    height=600
)

# Edit slider labels
fig['layout']['sliders'][0]['currentvalue']['prefix']='Paciente: '
for i in range(192):
    fig['layout']['sliders'][0]['steps'][i]['label']=i+1


fig.show()

# Escreve o gráfico em um arquivo html
#plotly.offline.plot(fig, filename='Chart_2'+'.html',auto_open=False)


In [ ]:
%%time

#from numpy import empty

Sinal_Original_IMP = np.empty((54, 7500))
IMF1_IMP = np.empty((54, 7500))
IMF2_IMP = np.empty((54, 7500))
IMF3_IMP = np.empty((54, 7500))
IMF4_IMP = np.empty((54, 7500))
IMF5_IMP = np.empty((54, 7500))
IMF6_IMP = np.empty((54, 7500))
IMF7_IMP = np.empty((54, 7500))

for i in range(54):
    
    if i == 0:
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        print(current_time)
    if i > 0 and i < 10:
        caminho5 = "C:\\Users\\ayalo\\PPG\\BIDMC-dadosOriginais\\bidmc_csv\\bidmc_0"+str(i)+"_Signals.csv"
        
        ####################################################
        # EMD - IMPEDANCIA
        signals = pd.read_csv(caminho5)
        s = signals[[' RESP']]
        s = s.to_numpy()
        s_trans = s.transpose()
        Signal_IMP = s_trans[0]
        time = np.linspace(0, 1, 7500)

        # Assign max IMF on signal
        emd1 = EMD()
        #emd1 = EMD(MAX_ITERATION = 10)

        # Execute EMD on signal
        #IMF = emd1.emd(Signal_PPG,time, max_imf = 3)
        IMF = emd1.emd(Signal_IMP[0:7500],time)

        Sinal_Original_IMP[i] = Signal_IMP[0:7500]
        IMF1_IMP[i] = IMF[0]
        IMF2_IMP[i] = IMF[1]
        IMF3_IMP[i] = IMF[2]
        IMF4_IMP[i] = IMF[3]
        IMF5_IMP[i] = IMF[4]
        IMF6_IMP[i] = IMF[5]
        IMF7_IMP[i] = IMF[6]        
                
    if i >= 10 and i<54:
        caminho5 = "C:\\Users\\ayalo\\PPG\\BIDMC-dadosOriginais\\bidmc_csv\\bidmc_"+str(i)+"_Signals.csv"
        
        ####################################################
        # EMD - IMPEDANCIA
        signals = pd.read_csv(caminho5)
        s = signals[[' RESP']]
        s = s.to_numpy()
        s_trans = s.transpose()
        Signal_IMP = s_trans[0]
        time = np.linspace(0, 1, 7500)

        # Assign max IMF on signal
        emd1 = EMD()
        #emd1 = EMD(MAX_ITERATION = 10)

        # Execute EMD on signal
        #IMF = emd1.emd(Signal_PPG,time, max_imf = 3)
        IMF = emd1.emd(Signal_IMP[0:7500],time)

        Sinal_Original_IMP[i] = Signal_IMP[0:7500]
        IMF1_IMP[i] = IMF[0]
        IMF2_IMP[i] = IMF[1]
        IMF3_IMP[i] = IMF[2]
        IMF4_IMP[i] = IMF[3]
        IMF5_IMP[i] = IMF[4]
        IMF6_IMP[i] = IMF[5]
        IMF7_IMP[i] = IMF[6]
        
        ####################################################

In [ ]:
%%time
## IMF PLOT
# Código necessário para executar o Plotly em uma "code cell"
configure_plotly_browser_state()

X = signals['Time [s]']
X = X[0:2000]

# Create figure
fig = make_subplots(rows=8, cols=1)
# Add all traces of pacient

for step in range(1,54):
    Y = Sinal_Original_IMP[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=1, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="black")
for step in range(1,54):
    Y = IMF1_IMP[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=2, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")
for step in range(1,54):
    Y = IMF2_IMP[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=3, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")
for step in range(1,54):
    Y = IMF3_IMP[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=4, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")
for step in range(1,54):
    Y = IMF4_IMP[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=5, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")
for step in range(1,54):
    Y = IMF5_IMP[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=6, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")
for step in range(1,54):
    Y = IMF6_IMP[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=7, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")
for step in range(1,54):
    Y = IMF7_IMP[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=8, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")

# Make 10th trace visible
fig.data[0].visible = True
fig.data[53].visible = True
fig.data[106].visible = True
fig.data[159].visible = True
fig.data[212].visible = True
fig.data[265].visible = True
fig.data[318].visible = True
fig.data[371].visible = True

# Create and add slider
steps = []
for i in range(53):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Padrão ouro dos pacientes: " + str(i+1)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][i+53] = True
    step["args"][0]["visible"][i+106] = True
    step["args"][0]["visible"][i+159] = True
    step["args"][0]["visible"][i+212] = True
    step["args"][0]["visible"][i+265] = True
    step["args"][0]["visible"][i+318] = True
    step["args"][0]["visible"][i+371] = True

    steps.append(step)

sliders = [dict(    
    steps=steps
)]

# Update xaxis properties
fig.update_xaxes(title_text="segundos (s)", row=8, col=1)


# Update yaxis properties
fig.update_yaxes(title_text="IMPEDANCIA", row=1, col=1)
fig.update_yaxes(title_text="IMF1", row=2, col=1)
fig.update_yaxes(title_text="IMF2", row=3, col=1)
fig.update_yaxes(title_text="IMF3", row=4, col=1)
fig.update_yaxes(title_text="IMF4", row=5, col=1)
fig.update_yaxes(title_text="IMF5", row=6, col=1)
fig.update_yaxes(title_text="IMF6", row=7, col=1)
fig.update_yaxes(title_text="IMF7", row=8, col=1)

fig.update_layout(
    title="EMD do sinal de Impedancia:",
    sliders=sliders,
    plot_bgcolor='rgba(0,0,0,0)',
    height=900
)

# Edit slider labels
fig['layout']['sliders'][0]['currentvalue']['prefix']='Paciente: '
for i in range(0,53):
    fig['layout']['sliders'][0]['steps'][i]['label']=i+1


fig.show()

# Escreve o gráfico em um arquivo html
plotly.offline.plot(fig, filename='Chart_3'+'.html',auto_open=False)

In [ ]:
%%time

# Number of sample points
N = 7500
# sample spacing
T = 1.0 / 125

XfftIMFO_IMP = np.empty((54, 3750))
YfftIMFO_IMP = np.empty((54, 3750))
XfftIMFA_IMP = np.empty((54, 3750))
YfftIMFA_IMP = np.empty((54, 3750))
XfftIMFP_IMP = np.empty((54, 3750))
YfftIMFP_IMP = np.empty((54, 3750))
XfftIMFU_IMP = np.empty((54, 3750))
YfftIMFU_IMP = np.empty((54, 3750))
ValorMedidoIMFO_IMP = np.array(range(54))
ValorMedidoIMFA_IMP = np.array(range(54))
ValorMedidoIMFP_IMP = np.array(range(54))
ValorMedidoIMFU_IMP = np.array(range(54))

for i in range(54):
    
    if i == 0:
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        print(current_time)
    if i > 0 and i < 54:

        ####################################################
        # FFT

        y0 = Sinal_Original_IMP[i]
        yf0 = fft(y0)
        x0 = fftfreq(N, T)[:N//2]
        y0 = 2.0/N * np.abs(yf0[0:N//2])
        XfftIMFO_IMP[i] = x0
        YfftIMFO_IMP[i] = y0
        
        y1 = IMF4_IMP[i]
        yf1 = fft(y1)
        x1 = fftfreq(N, T)[:N//2]
        y1 = 2.0/N * np.abs(yf1[0:N//2])
        XfftIMFA_IMP[i] = x1
        YfftIMFA_IMP[i] = y1
        
        y2 = IMF5_IMP[i]
        yf2 = fft(y2)
        x2 = fftfreq(N, T)[:N//2]
        y2 = 2.0/N * np.abs(yf2[0:N//2])
        XfftIMFP_IMP[i] = x2
        YfftIMFP_IMP[i] = y2
        
        y3 = IMF6_IMP[i]
        yf3 = fft(y3)
        x3 = fftfreq(N, T)[:N//2]
        y3 = 2.0/N * np.abs(yf3[0:N//2])
        XfftIMFU_IMP[i] = x3
        YfftIMFU_IMP[i] = y3
        
        ####################################################
        # Valor Medido
        
        z0 = y0[5:60].argmax()
        w0 = x0[z0+5]
        ValorMedidoIMFO_IMP[i] = w0*60
        
        z1 = y1[5:60].argmax()
        w1 = x1[z1+5]
        ValorMedidoIMFA_IMP[i] = w1*60
        
        z2 = y2[5:60].argmax()
        w2 = x2[z2+5]
        ValorMedidoIMFP_IMP[i] = w2*60
        
        z3 = y3[5:60].argmax()
        w3 = x3[z3+5]
        ValorMedidoIMFU_IMP[i] = w3*60
        ####################################################

In [ ]:
%%time
## FFT PLOT
# Código necessário para executar o Plotly em uma "code cell"
configure_plotly_browser_state()

# Create figure
fig = make_subplots(rows=4, cols=1)
# Add all traces of pacient
for step in range(1,54):
    X = XfftIMFO_IMP[step]
    Y = YfftIMFO_IMP[step]
    fig.add_scatter(x=X[0:500], y=Y[0:500],
                    row=1, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="gold")
for step in range(1,54):
    X = XfftIMFA_IMP[step]
    Y = YfftIMFA_IMP[step]
    fig.add_scatter(x=X[0:500], y=Y[0:500],
                    row=2, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")
for step in range(1,54):
    X = XfftIMFP_IMP[step]
    Y = YfftIMFP_IMP[step]
    fig.add_scatter(x=X[0:500], y=Y[0:500],
                    row=3, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="green")
for step in range(1,54):
    X = XfftIMFU_IMP[step]
    Y = YfftIMFU_IMP[step]
    fig.add_scatter(x=X[0:500], y=Y[0:500],
                    row=4, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="blue") 

# Make 10th trace visible
fig.data[0].visible = True
fig.data[53].visible = True
fig.data[106].visible = True
fig.data[159].visible = True


# Create and add slider
steps = []
for i in range(53):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Padrão ouro dos pacientes: " + str(i+1)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][i+53] = True
    step["args"][0]["visible"][i+106] = True
    step["args"][0]["visible"][i+159] = True
    steps.append(step)

sliders = [dict(
    steps=steps
)]

# Update xaxis properties
fig.update_xaxes(title_text="frequencia (Hz)", row=4, col=1)

# Update yaxis properties
fig.update_yaxes(title_text="IMPEDANCIA", row=1, col=1)
fig.update_yaxes(title_text="IMF5", row=2, col=1)
fig.update_yaxes(title_text="IMF6", row=3, col=1)
fig.update_yaxes(title_text="IMF7", row=4, col=1)

fig.update_layout(
    title="FFT do sinal de Impedancia:",
    sliders=sliders,
    plot_bgcolor='rgba(0,0,0,0)',
    height=600
)

# Edit slider labels
fig['layout']['sliders'][0]['currentvalue']['prefix']='Paciente: '
for i in range(0,53):
    fig['layout']['sliders'][0]['steps'][i]['label']=i+1


fig.show()

# Escreve o gráfico em um arquivo html
plotly.offline.plot(fig, filename='Chart_4'+'.html',auto_open=False)

In [ ]:
## PADRAO OURO
# Código necessário para executar o Plotly em uma "code cell"
configure_plotly_browser_state()

# Create figure
fig = make_subplots(rows=4, cols=1)

X = np.linspace(1, 1, 54)

# Add all traces of pacient
for i in range(1,54):
    X[i] = i
    
fig.add_scatter(x=X, y=ValorOuro, name = "(x,y)", marker_color="black",row=1, col=1, showlegend = False)
fig.add_scatter(x=X, y=ValorMedidoIMFO_IMP, name = "(x,y)", row=1, col=1,marker_color="gold", showlegend = False)
    
fig.add_scatter(x=X, y=ValorOuro, name = "(x,y)", marker_color="black", row=2, col=1, showlegend = False)
fig.add_scatter(x=X, y=ValorMedidoIMFA_IMP, name = "(x,y)", row=2, col=1,marker_color="red", showlegend = False)


fig.add_scatter(x=X, y=ValorOuro, name = "(x,y)", marker_color="black", row=3, col=1, showlegend = False)
fig.add_scatter(x=X, y=ValorMedidoIMFP_IMP, name = "(x,y)", row=3, col=1,marker_color="green", showlegend = False)

fig.add_scatter(x=X, y=ValorOuro, name = "(x,y)", marker_color="black", row=4, col=1, showlegend = False)
fig.add_scatter(x=X, y=ValorMedidoIMFU_IMP, name = "(x,y)", row=4, col=1, marker_color="blue", showlegend = False) 

# Make 10th trace visible
fig.data[0].visible = True

# Update xaxis properties
fig.update_xaxes(title_text="Respirações (rpm)", row=4, col=1)

# Update yaxis properties
fig.update_yaxes(title_text="IMPEDANCIA", row=1, col=1)
fig.update_yaxes(title_text="IMF5", row=2, col=1)
fig.update_yaxes(title_text="IMF6", row=3, col=1)
fig.update_yaxes(title_text="IMF7", row=4, col=1)

fig.update_layout(
    title="Valores da Frequencia respiratoria obtidos no sinal de Impedancia:",
    plot_bgcolor='rgba(0,0,0,0)',
    height=600
)

fig.show()

# Escreve o gráfico em um arquivo html
plotly.offline.plot(fig, filename='Chart_5'+'.html',auto_open=False)

In [ ]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

A = ValorOuro[1:54]
F = ValorMedidoIMFO_IMP[1:54]
G = ValorMedidoIMFA_IMP[1:54]
H = ValorMedidoIMFP_IMP[1:54]
I = ValorMedidoIMFU_IMP[1:54]
print(rmse(A, F))
print(rmse(A, G))
print(rmse(A, H))
print(rmse(A, I))

In [ ]:
%%time

#from numpy import empty
Sinal_Original = np.empty((54, 7500))
IMF1 = np.empty((54, 7500))
IMF2 = np.empty((54, 7500))
IMF3 = np.empty((54, 7500))
IMF4 = np.empty((54, 7500))
IMF5 = np.empty((54, 7500))
IMF6 = np.empty((54, 7500))
IMF7 = np.empty((54, 7500))

for i in range(54):
    
    if i == 0:
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        print(current_time)
    if i > 0 and i < 10:
        caminho5 = "C:\\Users\\ayalo\\PPG\\BIDMC-dadosOriginais\\bidmc_csv\\bidmc_0"+str(i)+"_Signals.csv"
        
        ###################################################
        # EMD
        # Define signal
        signals = pd.read_csv(caminho5)
        s = signals[[' PLETH']]
        s = s.to_numpy()
        s_trans = s.transpose()
        Signal_PPG = s_trans[0]
        time = np.linspace(0, 1, 7500)

        # Assign max IMF on signal
        emd1 = EMD()
        #emd1 = EMD(MAX_ITERATION = 10)

        # Execute EMD on signal
        #IMF = emd1.emd(Signal_PPG,time, max_imf = 3)
        IMF = emd1.emd(Signal_PPG[0:7500],time)

        Sinal_Original[i] = Signal_PPG[0:7500]
        IMF1[i] = IMF[0]
        IMF2[i] = IMF[1]
        IMF3[i] = IMF[2]
        IMF4[i] = IMF[3]
        IMF5[i] = IMF[4]
        IMF6[i] = IMF[5]
        IMF7[i] = IMF[6]

        ####################################################
                
    if i >= 10 and i<54:
        caminho5 = "C:\\Users\\ayalo\\PPG\\BIDMC-dadosOriginais\\bidmc_csv\\bidmc_"+str(i)+"_Signals.csv"
        
        ####################################################
        # EMD
        # Define signal
        signals = pd.read_csv(caminho5)
        s = signals[[' PLETH']]
        s = s.to_numpy()
        s_trans = s.transpose()
        Signal_PPG = s_trans[0]
        time = np.linspace(0, 1, 7500)     

        # Assign max IMF on signal
        emd1 = EMD()
        #emd1 = EMD(MAX_ITERATION = 10)

        # Execute EMD on signal
        #IMF = emd1.emd(Signal_PPG,time, max_imf = 3)
        IMF = emd1.emd(Signal_PPG[0:7500],time)

        Sinal_Original[i] = Signal_PPG[0:7500]
        IMF1[i] = IMF[0]
        IMF2[i] = IMF[1]
        IMF3[i] = IMF[2]
        IMF4[i] = IMF[3]
        IMF5[i] = IMF[4]
        IMF6[i] = IMF[5]
        IMF7[i] = IMF[6]     

        ####################################################

In [ ]:
%%time
## IMF PLOT
# Código necessário para executar o Plotly em uma "code cell"
configure_plotly_browser_state()

X = np.linspace(0, 1, 2000)
# Create figure
fig = make_subplots(rows=8, cols=1)
# Add all traces of pacient

for step in range(1,54):
    Y = Sinal_Original[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=1, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="black")
for step in range(1,54):
    Y = IMF1[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=2, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")
for step in range(1,54):
    Y = IMF2[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=3, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")
for step in range(1,54):
    Y = IMF3[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=4, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")
for step in range(1,54):
    Y = IMF4[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=5, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")
for step in range(1,54):
    Y = IMF5[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=6, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")
for step in range(1,54):
    Y = IMF6[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=7, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")
for step in range(1,54):
    Y = IMF7[step]
    fig.add_scatter(x=X, y=Y[0:2000],
                    row=8, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")

# Make 10th trace visible
fig.data[0].visible = True
fig.data[53].visible = True
fig.data[106].visible = True
fig.data[159].visible = True
fig.data[212].visible = True
fig.data[265].visible = True
fig.data[318].visible = True
fig.data[371].visible = True

# Create and add slider
steps = []
for i in range(53):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Padrão ouro dos pacientes: " + str(i+1)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][i+53] = True
    step["args"][0]["visible"][i+106] = True
    step["args"][0]["visible"][i+159] = True
    step["args"][0]["visible"][i+212] = True
    step["args"][0]["visible"][i+265] = True
    step["args"][0]["visible"][i+318] = True
    step["args"][0]["visible"][i+371] = True

    steps.append(step)

sliders = [dict(   
    steps=steps
)]

# Update xaxis properties
fig.update_xaxes(title_text="segundos (s)", row=8, col=1)


# Update yaxis properties
fig.update_yaxes(title_text="PPG", row=1, col=1)
fig.update_yaxes(title_text="IMF1", row=2, col=1)
fig.update_yaxes(title_text="IMF2", row=3, col=1)
fig.update_yaxes(title_text="IMF3", row=4, col=1)
fig.update_yaxes(title_text="IMF4", row=5, col=1)
fig.update_yaxes(title_text="IMF5", row=6, col=1)
fig.update_yaxes(title_text="IMF6", row=7, col=1)
fig.update_yaxes(title_text="IMF7", row=8, col=1)

fig.update_layout(
    title="EMD do sinal de PPG:",
    sliders=sliders,
    plot_bgcolor='rgba(0,0,0,0)',
    height=900
)

# Edit slider labels
fig['layout']['sliders'][0]['currentvalue']['prefix']='Paciente: '
for i in range(0,53):
    fig['layout']['sliders'][0]['steps'][i]['label']=i+1


fig.show()

# Escreve o gráfico em um arquivo html
plotly.offline.plot(fig, filename='Chart_6'+'.html',auto_open=False)

In [ ]:
%%time

# Number of sample points
N = 7500
# sample spacing
T = 1.0 / 125
XfftIMF5 = np.empty((54, 3750))
YfftIMF5 = np.empty((54, 3750))
XfftIMF6 = np.empty((54, 3750))
YfftIMF6 = np.empty((54, 3750))
XfftIMF7 = np.empty((54, 3750))
YfftIMF7 = np.empty((54, 3750))
ValorMedidoIMF5 = np.array(range(54))
ValorMedidoIMF6 = np.array(range(54))
ValorMedidoIMF7 = np.array(range(54))

for i in range(54):
    
    if i == 0:
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        print(current_time)
    if i > 0 and i < 54:
        
        ####################################################
        # FFT

        y1 = IMF5[i]
        yf1 = fft(y1)
        x1 = fftfreq(N, T)[:N//2]
        y1 = 2.0/N * np.abs(yf1[0:N//2])
        XfftIMF5[i] = x1
        YfftIMF5[i] = y1
        
        y2 = IMF6[i]
        yf2 = fft(y2)
        x2 = fftfreq(N, T)[:N//2]
        y2 = 2.0/N * np.abs(yf2[0:N//2])
        XfftIMF6[i] = x2
        YfftIMF6[i] = y2
        
        y3 = IMF7[i]
        yf3 = fft(y3)
        x3 = fftfreq(N, T)[:N//2]
        y3 = 2.0/N * np.abs(yf3[0:N//2])
        XfftIMF7[i] = x3
        YfftIMF7[i] = y3
        ####################################################
        # Valor Medido
        
        z1 = y1[5:64].argmax()
        w1 = x1[z1+5]
        ValorMedidoIMF5[i] = w1*60
        
        z2 = y2[5:64].argmax()
        w2 = x2[z2+5]
        ValorMedidoIMF6[i] = w2*60
        
        z3 = y3[5:64].argmax()
        w3 = x3[z3+5]
        ValorMedidoIMF7[i] = w3*60
        ####################################################

In [ ]:
%%time
## FFT PLOT
# Código necessário para executar o Plotly em uma "code cell"
configure_plotly_browser_state()

# Create figure
fig = make_subplots(rows=3, cols=1)
# Add all traces of pacient
for step in range(1,54):
    X = XfftIMF5[step]
    Y = YfftIMF5[step]
    fig.add_scatter(x=X[0:500], y=Y[0:500],
                    row=1, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="red")
for step in range(1,54):
    X = XfftIMF6[step]
    Y = YfftIMF6[step]
    fig.add_scatter(x=X[0:500], y=Y[0:500],
                    row=2, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="green")
for step in range(1,54):
    X = XfftIMF7[step]
    Y = YfftIMF7[step]
    fig.add_scatter(x=X[0:500], y=Y[0:500],
                    row=3, col=1, visible=False,
                    showlegend = False,
                    name = "(x,y)",
                    marker_color="blue") 

# Make 10th trace visible
fig.data[0].visible = True
fig.data[53].visible = True
fig.data[106].visible = True


# Create and add slider
steps = []
for i in range(53):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Padrão ouro dos pacientes: " + str(i+1)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][i+53] = True
    step["args"][0]["visible"][i+106] = True
    steps.append(step)

sliders = [dict(
    steps=steps
)]

# Update xaxis properties
fig.update_xaxes(title_text="Frequencia (Hz)", row=3, col=1)


# Update yaxis properties
fig.update_yaxes(title_text="IMF5", row=1, col=1)
fig.update_yaxes(title_text="IMF6", row=2, col=1)
fig.update_yaxes(title_text="IMF7", row=3, col=1)

fig.update_layout(
    title="FFT do sinal de PPG:",
    sliders=sliders,
    plot_bgcolor='rgba(0,0,0,0)',
    height=600
)

# Edit slider labels
fig['layout']['sliders'][0]['currentvalue']['prefix']='Paciente: '
for i in range(0,53):
    fig['layout']['sliders'][0]['steps'][i]['label']=i+1


fig.show()

# Escreve o gráfico em um arquivo html
plotly.offline.plot(fig, filename='Chart_7'+'.html',auto_open=False)

In [ ]:
## PADRAO OURO
# Código necessário para executar o Plotly em uma "code cell"
configure_plotly_browser_state()

# Create figure
fig = make_subplots(rows=3, cols=1)

X = np.linspace(1, 1, 54)

# Add all traces of pacient
for i in range(1,54):
    X[i] = i
    

fig.add_scatter(x=X, y=ValorOuro, name = "(x,y)", marker_color="black",row=1, col=1, showlegend = False)
fig.add_scatter(x=X, y=ValorMedidoIMF5, name = "(x,y)", row=1, col=1,marker_color="red", showlegend = False)


fig.add_scatter(x=X, y=ValorOuro, name = "(x,y)", marker_color="black",row=2, col=1, showlegend = False)
fig.add_scatter(x=X, y=ValorMedidoIMF6,name = "(x,y)", row=2, col=1,marker_color="green", showlegend = False)

fig.add_scatter(x=X, y=ValorOuro, name = "(x,y)", marker_color="black",row=3, col=1, showlegend = False)
fig.add_scatter(x=X, y=ValorMedidoIMF7, name = "(x,y)", row=3, col=1, marker_color="blue", showlegend = False)



  

# Make 10th trace visible
fig.data[0].visible = True

# Update xaxis properties
fig.update_xaxes(title_text="Respirações (rpm)", row=3, col=1)


# Update yaxis properties
fig.update_yaxes(title_text="IMF5", row=1, col=1)
fig.update_yaxes(title_text="IMF6", row=2, col=1)
fig.update_yaxes(title_text="IMF7", row=3, col=1)


fig.update_layout(
    title="Valores da Frequencia respiratoria obtidos no sinal de PPG:",
    plot_bgcolor='rgba(0,0,0,0)',
    height=600
)

fig.show()

# Escreve o gráfico em um arquivo html
plotly.offline.plot(fig, filename='Chart_8'+'.html',auto_open=False)

In [ ]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

A = ValorOuro[1:54]
F = ValorMedidoIMF5[1:54]
G = ValorMedidoIMF6[1:54]
H = ValorMedidoIMF7[1:54]
print(rmse(A, F))
print(rmse(A, G))
print(rmse(A, H))
